<a href="https://colab.research.google.com/github/tomonari-masada/course2024-intro2ml/blob/main/10_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# パーセプトロンとSVM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

## MNISTデータを使う

* scikit-learnに用意されている仕組みを利用してMNISTデータをダウンロードする。

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
X, y = mnist.data, mnist.target
print(X.shape)
print(y.shape)

* 通常、MNISTデータは最後の10000件をテストデータとして使う。

In [ ]:
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

* テストデータ以外の60000件を、訓練データ50000件と検証データ10000件へ分割しておく。

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,
                                                      test_size=10000,
                                                      random_state=42)

In [ ]:
print(X_train.shape, X_valid.shape, X_test.shape)

* 訓練データの最初の100個を可視化してみる。

In [ ]:
fig, axes = plt.subplots(10, 10)
for x, ax in zip(X_train.values, axes.ravel()):
  ax.axis('off') # 軸を消す
  ax.matshow(x.reshape(28, 28), cmap=plt.cm.gray)
plt.show()

* 問題を2値分類問題に変える。
  * 0とそれ以外の2値分類問題にしてしまう。

In [ ]:
y_train = (y_train != '0') * 1
y_valid = (y_valid != '0') * 1
y_test = (y_test != '0') * 1

In [ ]:
print(f'{(y_train == 0).sum()} zero digits and {(y_train == 1).sum()} non-zero digits in training set')

## 前処理：標準化
* 各ピクセルでの平均が0、標準偏差が1になるように、データを標準化しておく。
* こうすると分類精度が良くなったりする。

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train) # 訓練データで平均と標準偏差を計算
X_train = scaler.transform(X_train) # 訓練データで求めた平均と標準偏差を使って標準化する
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


## パーセプトロンによる分類の実行
* `sklearn.linear_model.Perceptron`クラスを使う。

## パーセプトロンでの正則化
* scikit-learnのパーセプトロンでは、`alpha`というパラメータを調整する。
  * リッジ回帰やLassoの`alpha`と同じ。大きいほど正則化が強く効く。
* 正則化の種類は`penalty`を`'l2'`か`'l1'`に設定することでおこなう。
  * `l2`だと、係数の二乗の和を最小化する。
  * `l1`だと、係数の絶対値の和を最小化する。

In [ ]:
from sklearn.linear_model import Perceptron

clf = Perceptron()
clf.fit(X_train, y_train)

In [ ]:
print(f'Accuracy: {clf.score(X_valid, y_valid):.4f}')

## SVMによる分類の実行
* ここでは、`LinearSVC`クラスを使う。
  * `SVC`クラスを使ってもよい。
  * `SVC`クラスを使うと、カーネルを指定できる。
* 分類器を準備し、訓練データを渡して係数と切片を推定させる。
  * 訓練データの個数が多いので、少し時間がかかる。
  * 推定計算が収束しない場合は、`max_iter`を増やしてみる。

## SVMでの正則化
* scikit-learnの`LinearSVC`では、`C`というパラメータを調整する。
* `C`は、小さいほど、正則化が強く効く＝係数をより強くゼロに近づける。
 * デフォルトの設定は`C=1.0`。
 * リッジ回帰やLassoの`alpha`とは、向きが逆であることに、注意。

In [ ]:
from sklearn.svm import SVC, LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train)

In [ ]:
print(f'Accuracy: {clf.score(X_valid, y_valid):.4f}')

* 推定された係数を可視化してみる。
  * 0とそれ以外を分類するとき、どのピクセルが有効かが見えるかも。

In [ ]:
plt.imshow(clf.coef_.reshape(28,28));

* 試行錯誤する。

In [ ]:
clf = LinearSVC(C=0.1, max_iter=10000)
clf.fit(X_train, y_train)
print(f'Accuracy: {clf.score(X_valid, y_valid):.4f}')

## 見つけ出した最善の設定を使ってテストデータ上で評価


In [ ]:
clf = LinearSVC()
clf.fit(X_train, y_train)
print(f'Accuracy: {clf.score(X_test, y_test):.4f}')

# 課題
* モデルとして`LinearSVC`を使ったとき、上の検証データでの正解率が最も良くなるパラメータ`C`の値を調べよう。
* その`C`の値を使って、テストデータでの正解率を求めよう。